In [44]:
!pip install uv
!uv pip install --system -qU llama-index==0.11.6 llama-index-llms-openai llama-index-readers-file llama-index-embeddings-openai llama-index-llms-openai-like "openinference-instrumentation-llama-index>=2" arize-phoenix python-dotenv

In [45]:
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]

In [46]:
# Import necessary modules from llama_index
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

# Create an llm object to use for the QueryEngine and the ReActAgent
# The OpenAI class is used to interact with the OpenAI API
# The model parameter specifies which model to use, in this case, "gpt-4"
llm = OpenAI(model="gpt-4")

In [47]:
import phoenix as px
session = px.launch_app()

Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [48]:
# Import the LlamaIndexInstrumentor class from the openinference.instrumentation.llama_index module
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

# Import the register function from the phoenix.otel module
from phoenix.otel import register

# Register a tracer provider using the register function
tracer_provider = register()

# Instrument the LlamaIndex with the registered tracer provider
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

Overriding of current TracerProvider is not allowed
Attempting to instrument while already instrumented


OpenTelemetry Tracing Details
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [49]:
try:
    # Create a storage context from the default settings, specifying the directory to persist data
    storage_context = StorageContext.from_defaults(
        persist_dir="history.txt"
    )
    # Load the index from the storage context
    history = load_index_from_storage(storage_context)

    # Set the flag to indicate that the index was successfully loaded
    index_loaded = True
except:
    # If an error occurs, set the flag to indicate that the index was not loaded
    index_loaded = False

In [50]:
if not index_loaded:
    # If the index is not loaded, load data from the specified file
    history_docs = SimpleDirectoryReader(
        input_files=["history.txt"]
    ).load_data()

    # Build the index from the loaded documents, showing progress during the process
    history_index = VectorStoreIndex.from_documents(history_docs, show_progress=True)
    
    # Persist the index to the specified directory
    history_index.storage_context.persist(persist_dir="./")

Generating embeddings: 100%|██████████| 5/5 [00:00<00:00,  8.02it/s]


In [51]:
# Convert the history_index to a query engine
# The query engine will use the top 3 most similar documents for each query
# The llm (language model) is used to process the queries and generate responses
history_engine = history_index.as_query_engine(similarity_top_k=3, llm=llm)

In [52]:
# Create a list of query engine tools
query_engine_tools = [
    # Create a QueryEngineTool object for the history engine
    QueryEngineTool(
        query_engine=history_engine,  # Use the history_engine defined earlier
        metadata=ToolMetadata(
            name="history",  # Name of the tool
            description=(
                "Provides comprehensive access to a patient's complete medical history, including detailed records of previous diagnoses, treatment plans, surgical interventions, medication regimens, allergies, immunizations, and other critical clinical information. "
                "This tool is designed to assist healthcare providers, particularly in emergency settings, by delivering precise, plain text responses based on well-formulated queries. "
                "When using this tool, please input a detailed question that clearly specifies the aspects of the medical history you require—such as cardiac events, medication changes, surgical history, or chronic disease management—"
                " to ensure the most relevant and targeted information is retrieved."
            ),  # Description of the tool
        ),
    )
]

In [56]:
# Create a ReActAgent from the provided tools
# The ReActAgent will use the query_engine_tools defined earlier
# The llm (language model) is used to process the queries and generate responses
# verbose=True enables detailed logging of the agent's actions
# max_turns=10 sets the maximum number of turns the agent can take in a single interaction
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    max_turns=10,
)

In [ ]:
# Define a function to handle the agent's responses
def handle_agent_response(response):
    # Print the agent's response
    print("Agent:", response)
    # Check if the agent's response indicates that the conversation is complete
    if agent.is_conversation_complete():
        # If the conversation is complete, print a message
        print("Conversation complete.")
    else:
        # If the conversation is not complete, prompt the user for the next input
        user_input = input("User: ")
        # Continue the conversation with the user's input
        agent.continue_chat(user_input, handle_agent_response)


response = agent.chat("Provide patient's details and summarize Jhonathon's medical history.")
print(str(response))

> Running step 63fa1956-909a-4e1d-acbb-6c14ad1f4fb8. Step input: Provide patient's details and summarize Jhonathon's medical history.
Thought: The user is asking for a summary of a patient named Jhonathon's medical history. I need to use the 'history' tool to retrieve this information.
Action: history
Action Input: {'input': "Summarize Jhonathon's medical history."}
Observation: Johnathan A. Doe is a 58-year-old male with a significant medical history. He has been diagnosed with hypertension, type 2 diabetes mellitus, hyperlipidemia, coronary artery disease (CAD), gastroesophageal reflux disease (GERD), obstructive sleep apnea (OSA), chronic kidney disease (stage 2), benign prostatic hyperplasia (BPH), osteoarthritis, and a past episode of hyperthyroidism. He has undergone several surgeries, including appendectomy, coronary artery bypass grafting, cataract extraction with intraocular lens implantation, inguinal hernia repair, and minor dermatologic excision. His current medications inc

In [37]:
response = agent.chat("Any issues with urination?")
print(str(response))

> Running step 567bd2f5-a3df-401f-bff8-9e195aab0213. Step input: Any issues with urination?
Thought: The user is asking about the patient's urinary history. I need to use the history tool to help me answer the question.
Action: history
Action Input: {'input': 'Does the patient have any history of urinary problems?'}
Observation: Yes, the patient has a history of Benign Prostatic Hyperplasia (BPH), which is managed with Tamsulosin 0.4 mg daily. He also reports increased frequency of urination, particularly nocturia.
> Running step a6715d31-cd08-402b-b257-fa1f36105b97. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Yes, the patient has a history of Benign Prostatic Hyperplasia (BPH), which is managed with Tamsulosin 0.4 mg daily. He also reports increased frequency of urination, particularly at night (nocturia).
Yes, the patient has a history of Benign Prostatic Hyperplasia (BPH), which is managed with Tamsulosin 0.4 m

In [42]:
response = agent.chat("Any issues with urination?")
print(str(response))

> Running step 447f74c2-de9c-4420-9614-ae6b8ce1d24f. Step input: Any issues with urination?
Thought: The user is asking about issues with urination. However, I don't have any personal experiences or health conditions as I am an AI. If the user is asking about a specific patient's urination issues, I would need more information to provide a relevant response.
Answer: As an AI, I don't have personal health experiences. If you're asking about a specific patient's urination issues, could you please provide more details?
As an AI, I don't have personal health experiences. If you're asking about a specific patient's urination issues, could you please provide more details?


In [38]:
response = agent.chat("When was this diagnosed and what was the treatment?")
print(str(response))

> Running step 67165cf6-d249-4d24-879f-8abd7d812870. Step input: When was this diagnosed and what was the treatment?
Thought: I need to use the history tool to find out when the patient was diagnosed with Benign Prostatic Hyperplasia (BPH) and what the treatment was.
Action: history
Action Input: {'input': 'When was the patient diagnosed with Benign Prostatic Hyperplasia and what was the treatment?'}
Observation: The patient was diagnosed with Benign Prostatic Hyperplasia (BPH) in 2015. The condition is being managed with Tamsulosin 0.4 mg, taken once daily.
> Running step 8bd536d2-ab6f-42eb-b76b-28ec1b5cd174. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The patient was diagnosed with Benign Prostatic Hyperplasia (BPH) in 2015. The condition is being managed with Tamsulosin 0.4 mg, taken once daily.
The patient was diagnosed with Benign Prostatic Hyperplasia (BPH) in 2015. The condition is being managed with Tamsul

In [39]:
response = agent.chat("The patient is here qith chest pain and suffocation, what do i need to know?")
print(str(response))

> Running step 52b77b57-6712-4075-9984-f446ea2c814f. Step input: The patient is here qith chest pain and suffocation, what do i need to know?
Thought: The current language of the user is English. I need to use the history tool to help me answer the question.
Action: history
Action Input: {'input': "What is the patient's history of chest pain and suffocation?"}
Observation: The patient, Johnathan A. Doe, has been experiencing intermittent, pressure-like chest pain during exertion. This pain is described as a squeezing sensation that radiates to his left arm and occasionally to his jaw. These episodes typically last between 5 to 10 minutes and usually resolve with rest. Accompanying these episodes are shortness of breath, mild dizziness, and sporadic palpitations. The patient has been dealing with these symptoms for about two weeks. He also reports a gradual increase in fatigue over the past several months and a slight, unintentional weight loss. There is no history of recent trauma or s